In [1]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_from_disk

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [2]:
people_dataset = load_from_disk('../datasets/intermediate/people_dataset')

In [3]:
people_dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'corrupt_people', 'not_corrupt_people'],
        num_rows: 750
    })
    train: Dataset({
        features: ['text', 'corrupt_people', 'not_corrupt_people'],
        num_rows: 11417
    })
    validation: Dataset({
        features: ['text', 'corrupt_people', 'not_corrupt_people'],
        num_rows: 1448
    })
})

In [4]:
people_dataset['test'][0]

{'text': 'Gyurcsány Ferencet és Jánosi Györgyöt november 12-ére, Szilvásy Györgyöt és Benedek Andrást november 16-ára, Mesterházy Attilát és Keszthelyi Andrást november 18-ára idézi be a Zuschlag-perben tanúkénti meghallgatásra a Bács-Kiskun Megyei Bíróság - közölte Fodor Endre tanácsvezető bíró a hétfői tárgyalási napon Kecskeméten.\nKözlése szerint a szeptemberben és októberben lefolytatott bizonyítási eljárás alapján ezeket a személyeket, akiknek vélhetően tudomásuk van a vád tárgyává tett cselekményről, legalábbis annak egy részéről, indokolt meghallgatni.\nGyurcsány Ferenc és Jánosi György a perben tárgyalt időszakban, illetve annak egy részében ifjúsági és sportminiszter, Szilvásy György, Benedek András és Mesterházy Attila államtitkár, Keszthelyi András pedig miniszteri kabinetfőnök volt.\nA bíróság hétfőn részben helyt adott Zuschlag János elsőrendű vádlott indítványának. Elutasította a Baloldali Ifjúsági Társulás (BIT) minden tagjának a meghallgatására tett indítványát, viszon

In [5]:
people_dataset = people_dataset.map(lambda row: {'all': row['corrupt_people'] + row['not_corrupt_people']})

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Map:   0%|          | 0/11417 [00:00<?, ? examples/s]

Map:   0%|          | 0/1448 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from transformers import BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-v0.1"
peft_model_id = '../finetune/qlora-out-entity-extraction.backup/'

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    ),
)
model = PeftModel.from_pretrained(model, peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CUDA extension not installed.
CUDA extension not installed.


In [8]:
from tqdm import tqdm

device = 'cuda:0'
# features: ['text', 'corrupt_institutions', 'not_corrupt_institutions', 'all'],
people_classification_prompt = '''[személy klasszifikáció]
{text}

###

összes: {all}
korrupcióban érintett:'''

tp = 0
fp = 0
tn = 0
fn = 0

for row in tqdm(people_dataset['test']):
    text = row['text']
    corrupt_people = row['corrupt_people']
    not_corrupt_people = row['not_corrupt_people']
    all_people = row['all']
    input_text = people_classification_prompt.format(all=', '.join(all_people), text=text)
    with torch.cuda.amp.autocast():
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        if len(inputs['input_ids'][0]) > 2048:
            continue
        outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=200, penalty_alpha=0.6, top_k=3)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        sresult = result[result.rfind(':')+2:].strip()
        output_labels = sresult.strip().split(', ')
        for people in corrupt_people:
            if people in output_labels:
                tp += 1
            else:
                fn += 1
        for people in not_corrupt_people:
            if people in output_labels:
                fp += 1
            else:
                tn += 1
        print(tp, tn, fp, fn)

  0%|                                                                                                                                                     | 0/750 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
2023-12-02 10:58:53.285835: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 10:58:53.285897: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 10:58:53.287217: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register 

5 3 0 1


  0%|▎                                                                                                                                          | 2/750 [00:29<2:44:21, 13.18s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


6 5 0 2


  0%|▌                                                                                                                                          | 3/750 [00:36<2:12:02, 10.61s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

660 1001 98 191


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 748/750 [1:04:57<00:10,  5.11s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


661 1004 98 191


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [1:05:02<00:00,  5.20s/it]

662 1006 98 191


In [9]:
print('precision', tp/(tp+fp))
print('recall', tp/(tp+fn))
print('accuracy', (tp+tn)/(tp+tn+fp+fn))

precision 0.8710526315789474
recall 0.776084407971864
accuracy 0.8523249872253449


In [10]:
all_possible_labels = set()
for labels in truths:
    for label in labels:
        all_possible_labels.add(label)
for labels in preds:
    for label in labels:
        all_possible_labels.add(label)
label_to_id = {key: value for value, key in enumerate(all_possible_labels)}
label_to_id

NameError: name 'truths' is not defined

In [ ]:
i_preds = [[1 if value in preds_a else 0 for value in label_to_id] for preds_a in preds]

In [ ]:
i_preds

In [ ]:
i_truths = [[1 if value in truths_a else 0 for value in label_to_id] for truths_a in truths]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(i_truths, i_preds, target_names=label_to_id))

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(i_truths, i_preds)
acc = accuracy_score(labels, preds)
{
    'accuracy': acc,
    'precision': precision,
    'recall': recall,
    'f1': f1
}

In [ ]:
import numpy as np
y_true = np.array([[0, 1, 1, 1],[0,0,1,0],[1,1,0,0]])
y_scores = np.array([[0.2, 0.6, 0.1, 0.8],[0.4,0.9,0.8,0.6],[0.8,0.4,0.5,0.7]])
threshold = 0.5
y_pred=[]
for sample in  y_scores:
  y_pred.append([1 if i>=0.5 else 0 for i in sample ] )
y_pred = np.array(y_pred)
y_pred


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(i_truths, i_preds)


In [ ]:
print("Actual \n", y_true)
print("\nPredicted \n",y_pred)


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(i_truths, i_preds)
